In [15]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [16]:
f = h5py.File('../../data/cube_test.h5', 'r')
vertices = f['vertices'][:]
edges = f['edges'][:]
radius = f['radius'][()]
images = f['image']
channels = list(images.keys())

In [17]:
dna1c = images['DNA1']

In [92]:
def get_voxels_within_radius(center, radius, shape):
    arr = np.zeros(shape)

    # create a meshgrid of indices
    z, y, x = np.meshgrid(np.arange(arr.shape[0]), np.arange(arr.shape[1]), np.arange(arr.shape[2]), indexing='ij')
    indices = np.stack([z, y, x], axis=-1)

    # calculate the Euclidean distance between each voxel and the center
    distances = np.linalg.norm(indices - center, axis=-1)

    # get the boolean array indicating which voxels are within the radius from the center
    within_radius = distances <= radius

    # get the indices of the voxels that are within the radius from the center
    indices_within_radius = indices[within_radius]
    
    return indices_within_radius
    
def calculate_angles(voxels, center):
    # subtract the center from the voxels and ignore the first dimension
    vecs = voxels[:, 1:] - center[1:]
    # calculate the angles using arctan2
    angles = np.arctan2(vecs[:, 0], vecs[:, 1])
    angles_percent = ((angles / (2 * np.pi)) + 0.5) * 100
    return angles_percent

def get_polarization(channel, radius, threshold):
    with h5py.File('../../data/cube_test.h5', 'r') as f:
        centers = f['vertices']
        cube = f['image'][channel][:]
        cube = np.where(cube > threshold, 1, 0)
        regions = np.zeros((len(centers),10))
        for i, center in enumerate(centers):
            voxels = get_voxels_within_radius(center, radius, cube.shape)
            angles = calculate_angles(voxels, center)
            
            for ri, a in enumerate(range(0, 100, 10)):
                for j, angle in enumerate(angles):
                    if a <= angle < a + 10:
                        regions[i,ri] += cube[voxels[j][0], voxels[j][1], voxels[j][2]]
                
    return regions


In [94]:
get_polarization('HLAA', 10, 0.1)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 5.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [95]:
f.close()